In [0]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import SVG
import os
import pydotplus

import keras
from keras import regularizers
from keras.optimizers import Adam, SGD, rmsprop
from keras.models import Sequential,Model
from keras.layers import Dense, Activation, Dropout, Input, Dropout, concatenate
from keras.layers.recurrent import GRU, LSTM
from keras.utils import plot_model
# keras.utils.vis_utils.pydot = pydot
from keras.utils.vis_utils import model_to_dot

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve

%matplotlib inline

In [0]:
data = np.load("drive/My Drive/dl_mimic/imputed-normed-ep_1_24.npz", allow_pickle=True)

In [0]:
lst = data.files
for item in lst:
    print(item)
#     print(data[item])
data["ep_data"].shape

ep_data
ep_tdata
ep_data_masking
ep_tdata_masking
y_icd9
y_mor
adm_features_all
adm_labels_all
y_los


(36093, 288)

In [0]:
data["ep_tdata"].shape

(36093, 24, 15)

In [0]:
data['adm_features_all'].shape

(36093, 5)

In [0]:
data["adm_labels_all"].shape

(36093, 6)

In [0]:
# static features
df = pd.DataFrame(data=data['adm_features_all'], columns=["age", "aids", "hem", "mets", "admission_type"])

In [0]:
df

,age,aids,hem,mets,admission_type
0,26962.16444,0.0,0.0,0.0,2.0
1,22206.71360,0.0,0.0,0.0,2.0
2,19912.99384,0.0,0.0,1.0,0.0
3,7852.70300,0.0,0.0,0.0,0.0
4,24734.18824,0.0,0.0,0.0,2.0
...,...,...,...,...,...
36088,31246.45310,0.0,0.0,0.0,0.0
36089,22104.44584,0.0,0.0,0.0,0.0
36090,21289.95618,0.0,0.0,0.0,0.0
36091,25263.78914,0.0,0.0,0.0,2.0


In [0]:
# mortality labels
df2 = pd.DataFrame(data=data["adm_labels_all"], columns=["ih", "1-day", "2-day", "3-day", "30-day", "1-year"])

In [0]:
df2

,ih,1-day,2-day,3-day,30-day,1-year
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
36088,0,0,0,0,0,0
36089,0,0,0,0,0,0
36090,0,0,0,0,0,0
36091,0,0,0,0,0,0


In [0]:
result = pd.concat([df, df2], axis=1, sort=False)

In [0]:
result

,age,aids,hem,mets,admission_type,ih,1-day,2-day,3-day,30-day,1-year
0,26962.16444,0.0,0.0,0.0,2.0,0,0,0,0,0,0
1,22206.71360,0.0,0.0,0.0,2.0,0,0,0,0,0,0
2,19912.99384,0.0,0.0,1.0,0.0,0,0,0,0,0,0
3,7852.70300,0.0,0.0,0.0,0.0,0,0,0,0,0,0
4,24734.18824,0.0,0.0,0.0,2.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
36088,31246.45310,0.0,0.0,0.0,0.0,0,0,0,0,0,0
36089,22104.44584,0.0,0.0,0.0,0.0,0,0,0,0,0,0
36090,21289.95618,0.0,0.0,0.0,0.0,0,0,0,0,0,0
36091,25263.78914,0.0,0.0,0.0,2.0,0,0,0,0,0,0


In [0]:
((result["ih"]== 1) & (result["admission_type"]== 1)).sum()

112

In [0]:
result.loc[21280]

age               32502.88558
aids                  0.00000
hem                   0.00000
mets                  0.00000
admission_type        2.00000
ih                    1.00000
1-day                 0.00000
2-day                 0.00000
3-day                 0.00000
30-day                1.00000
1-year                1.00000
Name: 21280, dtype: float64

In [0]:
count = df["admission_type"].value_counts() 
count

0.0    25082
2.0     6226
1.0     4785
Name: admission_type, dtype: int64

In [0]:
np.mean(df["age"])

27277.54093426374

In [0]:
features_sequence = data['ep_tdata'].astype(np.float32) # 15 time series features
features_static = data['adm_features_all'].astype(np.float32) # 5 non-ts features,the unit of age: day
labels = data["adm_labels_all"][:, 3].astype(np.float32) # 48hrs or In-hospital  mortality, change here

In [0]:
count = 0
for i in range(labels.shape[0]):
    if labels[i] == 1:
        count += 1
count

1077